In [2]:
import sys, os, io, json, numpy as np, random
#from sklearn.metrics import roc_auc_score
#import xgboost as xgb
import tensorflow as tf
import pandas as pd
from split import split_graph, split_interface, make_split
import graphviz
from sklearn import metrics

/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.4_3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
pool = np.load('../prepare_dataset/pool_00.npz')
features_bool, f_time, label = (pool[name] for name in ['features', 'f_time', 'label'])

In [4]:
features = np.random.normal(loc=features_bool*1.0, scale=1.0)

In [5]:
#features_swap = features.copy()
#features_swap[:,0],features_swap[:,7] = features_swap[:,7],features_swap[:,0]
features_swap = np.concatenate([features[:,7:8], features[:,1:7], features[:,0:1], features[:,8:]], axis=1)

In [6]:
%%time
make_split(np.zeros(label.shape), features, label)

CPU times: user 5.4 s, sys: 805 ms, total: 6.21 s
Wall time: 2.22 s


{'avg_current_loss': array([0.69314718]),
 'best_avg_loss': 0.6124091340894955,
 'best_delta_down': -0.8576827683949451,
 'best_delta_up': -0.7922214464000513,
 'best_feature_index': 0,
 'best_index': 249197,
 'best_loss': 183722.74022684863,
 'current_loss': array([207944.15416803]),
 'thr': 1.2566736700329577}

In [7]:
%%time
make_split(np.zeros(label.shape), features_swap, label)

CPU times: user 5.36 s, sys: 563 ms, total: 5.92 s
Wall time: 2.28 s


{'avg_current_loss': array([0.69314718]),
 'best_avg_loss': 0.6124091340894955,
 'best_delta_down': -0.8576827683949451,
 'best_delta_up': -0.7922214464000513,
 'best_feature_index': 7,
 'best_index': 249197,
 'best_loss': 183722.74022684863,
 'current_loss': array([207944.15416803]),
 'thr': 1.2566736700329577}

In [8]:
class EMatrix:
    def __init__(self, features, label, bias=None):
        self.bias = bias
        self.features = features
        self.label = label

In [9]:
class LeafData:
    def __init__(self, info):
        self.val = info['prediction'] * info['learning_rate']
        
    def to_text(self, floatformat = '.6f'):
        return ('{:'+ floatformat + '}').format(self.val)
    
    def shape(self):
        return 'box'

class SplitData:
    def __init__(self, val):
        self.val = val
        
    def to_text(self, floatformat = '.4f'):
        return ('f_{{{ind}}} < {thr:'+floatformat+'}').format(ind=self.val['best_feature_index'], thr=self.val['thr'])
    
        
class TreeNode:
    def __init__(self):
        self.left = None
        self.right = None
        self.depth = 0
        self.val = None
        self.id = None
        
    def to_text(self, floatformat = '.6f'):
        return self.val.to_text(floatformat)
    
    def shape(self):
        return 'circle'
        

In [10]:
def init_id_helper(node, current_id):
    node.id = current_id[0]
    current_id[0] += 1
    if not isinstance(node, TreeNode):
        return
    init_id_helper(node.left, current_id)
    init_id_helper(node.right, current_id)

def init_id(root):
    current_id = [0]
    init_id_helper(root, current_id)
    return current_id[0]

In [11]:
def init_arrays_helper(node, arrays):
    if not isinstance(node, TreeNode):
        arrays['is_leaf'][node.id] = 1
        arrays['leaf_data'][node.id, 0] = node.val  # Leaf
        return
    init_arrays_helper(node.left, arrays)
    init_arrays_helper(node.right, arrays)
    arrays['yes_node'][node.id] = node.left.id
    arrays['no_node'][node.id] = node.right.id
    arrays['thresholds'][node.id] = node.val.val['thr']
    arrays['features'][node.id] = node.val.val['best_feature_index']
    arrays['is_leaf'][node.id] = 0
    arrays['depths'][node.id] = node.depth
    
def init_arrays(root, n):
    def empty_array():
        return np.zeros(n, dtype=np.int32)
    arrays = dict(features=empty_array(),
                  thresholds=np.zeros(n, dtype=np.float32),
                  yes_node=empty_array(),
                  no_node=empty_array(),
                  is_leaf=empty_array(),
                  depths=empty_array(),
                  leaf_data=np.zeros((n,1), dtype=np.float32)
                 )
    init_arrays_helper(root, arrays)
    arrays['treedepth'] = np.max(arrays['depths'])
    return arrays

In [12]:
def prior_finish(params, info, parent):
    if parent is None:
        return False
    return params['max_depth'] <= parent.depth

def post_finish(params, info, split_info, parent):
    if split_info['left_info']['ematrix'].label.shape[0] < 2:
        return True
    if split_info['right_info']['ematrix'].label.shape[0] < 2:
        return True
    return False

In [13]:
def split_ematrix(ematrix):
    split_info = make_split(ematrix.bias, ematrix.features, ematrix.label)
    thr = split_info['thr']
    features = ematrix.features
    bias = ematrix.bias
    label = ematrix.label
    best_feature = features[:, split_info['best_feature_index']] < thr
    best_feature_not = np.logical_not(best_feature)
    left_ematrix = EMatrix(features[best_feature, :], label[best_feature], bias[best_feature])
    right_ematrix = EMatrix(features[best_feature_not, :], label[best_feature_not], bias[best_feature_not])
    left_info = {'prediction': split_info['best_delta_up'], 'ematrix': left_ematrix}
    right_info = {'prediction': split_info['best_delta_down'], 'ematrix': right_ematrix}
    split_info['left_info'] = left_info
    split_info['right_info'] = right_info
    return left_info, right_info, split_info

In [14]:
def build_tree_helper(params, info, parent):
    info['learning_rate'] = params['learning_rate']
    if parent and parent.depth < 6:
        print("{d}".format(d=parent.depth) 
              if parent else '---',
              "".format(shape=info['ematrix'].label.shape[0]),
              end=' ', file=sys.stderr)
    if prior_finish(params, info, parent):
        return LeafData(info)
    left_info, right_info, split_info = split_ematrix(info['ematrix'])
    if post_finish(params, info, split_info, parent):
        #print(split_info['right_info']['ematrix'].label.shape[0])
        #print(split_info)
        return LeafData(info)
    node = TreeNode()
    node.depth = parent.depth + 1 if parent else 1
    node.val = SplitData(split_info)
    node.left = build_tree_helper(params, left_info, node)
    node.right = build_tree_helper(params, right_info, node)  
    return node

In [15]:
def build_tree(params, ematrix):
    info = {'ematrix': ematrix}
    return build_tree_helper(params, info=info, parent=None)

In [16]:
features.shape, label.shape

((300000, 15), (300000, 1))

In [17]:
start_params = {'max_depth': 4, 'learning_rate':5}

tree = build_tree(start_params, EMatrix(features, label, np.zeros(label.shape)))

1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  

In [18]:
tree.left.left.left.left.val, tree.val

(0.4, <__main__.SplitData at 0x1016d7550>)

In [19]:
def tree2gv(tree):
    result = graphviz.Graph('ni')
    #result.attr(size='12,0')
    tree2gv_helper(tree, result, '')
    return result

In [20]:
def tree2gv_helper(node, result, id):
    idn = id
    result.node(idn, node.to_text(), shape='box') # node.shape())
    if isinstance(node, LeafData):
        return
    if node.left is not None:
        idl = id + '0'
        tree2gv_helper(node.left, result, idl)
        result.edge(idn, idl)
    if node.right is not None:
        idr = id + '1'
        tree2gv_helper(node.right, result, idr)
        result.edge(idn, idr)
    
    

In [21]:
#tree2gv(tree)

In [22]:
tree

In [23]:
tree_arrays = init_arrays(tree, init_id(tree))

In [24]:
#tree_arrays

In [25]:
def tree_apply(tree_arrays, features):
    qi = np.zeros(features.shape[0], dtype=np.int32)
    for current_depth in range(tree_arrays['treedepth']):
        fi = tree_arrays['features'][qi]
        f = np.choose(fi, features.T)
        t = tree_arrays['thresholds'][qi]
        #print(qi, fi, f, t)
        #if current_depth == 0: 
        #    print(fi, f.shape, features.shape, f)
        answer = (f < t)*1
        new_qi = answer*tree_arrays['yes_node'][qi] + (1-answer)*tree_arrays['no_node'][qi]
        qi = new_qi
    leaf_data = tree_arrays['leaf_data'][qi, 0]
    return leaf_data

In [42]:
class EBooster:
    def __init__(self, forest):
        self.forest = forest
    
    def predict(self, features):
        pred = np.zeros(features.shape[0], dtype=np.float32)
        for tree, tree_arrays in self.forest:
            pred = pred + tree_apply(tree_arrays, features)
        return pred 
    
def train(params, ematrix, num_boost_round = 10):
    start_params = {'max_depth': 5, 'learning_rate': 0.3} 
    start_params.update(params)
    
    forest = []
    bias = np.zeros(ematrix.label.shape)
    for r in range(num_boost_round):
        print("\n{} round".format(r), file=sys.stderr)
        tree = build_tree(start_params, EMatrix(features, label, bias))
        print("tree ok, bias shape = {}".format(bias.shape), file=sys.stderr)
        tree_arrays = init_arrays(tree, init_id(tree))
        bias_delta = tree_apply(tree_arrays, features)
        print("apply ok, bias delta shape = {}".format(bias_delta.shape), file=sys.stderr)
        bias = bias + np.reshape(bias_delta, newshape=bias.shape)
        forest.append((tree, tree_arrays))
        print("forest appended", file=sys.stderr)
        
    return EBooster(forest)

In [30]:
pred = tree_apply(tree_arrays, features)

In [31]:
metrics.roc_auc_score(label[:, 0], pred)

0.5162252399236659

In [32]:
metrics.roc_auc_score(label[:, 0], np.zeros(label.shape[0]))

0.5

In [46]:
boost_params = {'max_depth': 4, 'learning_rate':0.3}
booster = train(boost_params, EMatrix(features, label), num_boost_round=12)


0 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  tree ok, bias shape = (300000, 1)
apply ok, bias delta shape = (300000,)
forest appended

1 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  tree ok, bias shape = (300000, 1)
apply ok, bias delta shape = (300000,)
forest appended

2 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  tree ok, bias shape = (300000, 1)
apply ok, bias delta shape = (300000,)
forest appended

3 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  2  3  4  4  3  4  4  tree ok, bias shape = (300000, 1)
apply ok, bias delta shape = (300000,)
forest appended

4 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  tree ok, bias shape = (300000, 1)
apply ok, bias delta shape = (300000,)
forest appended

5 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  

In [47]:
pred2 = booster.predict(features)

In [45]:
metrics.roc_auc_score(label[:, 0], pred2)

0.5241520744012362